In [ ]:
import torchvision as tv
import pandas as pd
import numpy as np
import torch
from torch import nn
import time

In [ ]:
BATCH_SIZE=256

In [ ]:
train_dataset = tv.datasets.FashionMNIST('.', train=True, transform=tv.transforms.ToTensor(), download=True)
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform=tv.transforms.ToTensor(), download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
model = torch.nn.Sequential(
    nn.Conv2d(1, 20, kernel_size=5, padding=2),
    nn.Tanh(),
    nn.AvgPool2d(2, stride=2),
    nn.Conv2d(20, 50, kernel_size=5, padding=2),
    nn.Tanh(),
    nn.AvgPool2d(2, stride=2),
    nn.Flatten(),
    nn.Linear(2450, 84),
    nn.Tanh(),
    nn.Linear(84, 10)
)

In [ ]:
model = model.to(device)

In [ ]:
from torchsummary import summary

summary(model, input_size=(1, 28, 28), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 28, 28]             520
              Tanh-2           [-1, 20, 28, 28]               0
         AvgPool2d-3           [-1, 20, 14, 14]               0
            Conv2d-4           [-1, 50, 14, 14]          25,050
              Tanh-5           [-1, 50, 14, 14]               0
         AvgPool2d-6             [-1, 50, 7, 7]               0
           Flatten-7                 [-1, 2450]               0
            Linear-8                   [-1, 84]         205,884
              Tanh-9                   [-1, 84]               0
           Linear-10                   [-1, 10]             850
Total params: 232,304
Trainable params: 232,304
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.46
Params size (MB): 0.89
Estimated T

In [ ]:
def evaluate_accuracy(data_iter, net, device):
    acc_sum, n = torch.Tensor([0]).to(device), 0
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [ ]:
def train(net, train_iter, test_iter, optimizer, num_epochs, device):
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

        test_acc = evaluate_accuracy(test_iter, net, device)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}' \
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

        if test_acc > 0.9:
          break

In [ ]:
lr, num_epochs = 0.001, 20
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, train_iter, test_iter, trainer, num_epochs, device)

epoch 1, loss 0.0025, train acc 0.784, test acc 0.828, time 12.4 sec
epoch 2, loss 0.0016, train acc 0.855, test acc 0.851, time 12.1 sec
epoch 3, loss 0.0014, train acc 0.870, test acc 0.861, time 13.1 sec
epoch 4, loss 0.0013, train acc 0.880, test acc 0.871, time 12.3 sec
epoch 5, loss 0.0012, train acc 0.888, test acc 0.877, time 12.1 sec
epoch 6, loss 0.0012, train acc 0.893, test acc 0.878, time 12.3 sec
epoch 7, loss 0.0011, train acc 0.898, test acc 0.878, time 12.3 sec
epoch 8, loss 0.0011, train acc 0.902, test acc 0.884, time 12.1 sec
epoch 9, loss 0.0010, train acc 0.905, test acc 0.884, time 12.2 sec
epoch 10, loss 0.0010, train acc 0.909, test acc 0.886, time 12.4 sec
epoch 11, loss 0.0009, train acc 0.912, test acc 0.887, time 12.3 sec
epoch 12, loss 0.0009, train acc 0.917, test acc 0.889, time 12.2 sec
epoch 13, loss 0.0009, train acc 0.918, test acc 0.890, time 12.1 sec
epoch 14, loss 0.0008, train acc 0.922, test acc 0.891, time 12.2 sec
epoch 15, loss 0.0008, train 